In [6]:
from langchain_groq import ChatGroq
import os
import config
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = config.groq_key
# os.environ["HTTP_PROXY"] = config.proxy
# os.environ["HTTPS_PROXY"] = config.proxy

In [7]:
human_prompt = HumanMessagePromptTemplate.from_template('Make up a funny company name from a company that makes: {product}')
chat_prompt_template = ChatPromptTemplate.from_messages([human_prompt])
llm = ChatGroq(model='llama3-8b-8192', temperature=0)

In [8]:
from langchain.chains import LLMChain

In [9]:
chain = chat_prompt_template | llm

In [10]:
print(chain.invoke(input= {'product': 'Computers'}).content)

APIConnectionError: Connection error.

## Sequential Chain

In [16]:
from langchain.chains import LLMChain, SimpleSequentialChain

In [17]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0)

### Chain Functionality

Topic for Blog Pots ----> [[Outline ----> Create Blog Post]] ----> Blog Post Text   
                          

In [25]:
template = "Give me a simple bullet point for a blog post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template)

chain_1 = LLMChain(llm=llm, prompt=first_prompt)
# print(chain_1.invoke(input={"topic": "LLM"})["text"])

template_2 = "Write a blog post using this outline {outline}"
second_prompt = ChatPromptTemplate.from_template(template_2)
chain_2 = LLMChain(llm=llm, prompt=second_prompt)


In [28]:
full_chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)
result = full_chain.invoke('LLM')



> Entering new SimpleSequentialChain chain...
Here's a simple bullet point for a blog post on Large Language Models (LLMs):

**What are Large Language Models (LLMs)?**

• **Powerful AI models**: LLMs are artificial intelligence models that are trained on vast amounts of text data to generate human-like language outputs.
• **Capable of understanding context**: Unlike traditional language models, LLMs can understand the context and nuances of language, allowing them to generate more accurate and relevant responses.
• **Applications in various industries**: LLMs have the potential to revolutionize industries such as customer service, content creation, and language translation, among others.
• **Limitations and challenges**: Despite their capabilities, LLMs also have limitations and challenges, such as bias, accuracy, and explainability, which need to be addressed.

This bullet point provides a brief overview of what LLMs are, their capabilities, and their potential applications, as well

## Sequential Chain

- Gives inputs, outputs and all the required outlines
- Is more customizable than SimpleSequentialChain

In [29]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [30]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0)

In [32]:
temp_1 = "Give a summary of this employee's performance review \n{review}"
prompt_1 = ChatPromptTemplate.from_template(temp_1)
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="summary")

temp_2 = "Extract the weaknesses from the given summary: \n{summary}"
prompt_2 = ChatPromptTemplate.from_template(temp_2)
chain_2 = LLMChain(llm=llm, prompt = prompt_2, output_key="weakness")

temp_3 = "Suggest an improvement plan for the given weaknesses: \n{weakness}"
prompt_3 = ChatPromptTemplate.from_template(temp_3)
chain_3 = LLMChain(llm=llm, prompt = prompt_3, output_key="plan")



In [33]:
seq_chain = SequentialChain(chains = [chain_1, chain_2, chain_3],
                            input_variables = ["review"],
                            output_variables = ['summary', 'weakness', 'plan'],
                            verbose=True)

In [34]:
review = '''Employee Name: John Doe
Position: Sales Representative
Review Period: Q3 2024

Strengths:
Strong Communication Skills: John consistently demonstrates excellent verbal and written communication. His ability to convey ideas clearly to both clients and team members has helped improve client relationships and close sales deals effectively.

Customer-Focused Approach: John shows a deep understanding of customer needs and provides tailored solutions that result in high customer satisfaction. His dedication to addressing customer concerns and following up on issues promptly is commendable.

Goal-Oriented: He is highly motivated and sets clear goals for himself. John has successfully met or exceeded his sales targets in the past three quarters, showing his dedication to achieving measurable results.

Team Collaboration: John works well with his colleagues and contributes positively to team projects. He is always willing to share his knowledge and mentor newer team members, which fosters a supportive team environment.

Weaknesses:
Time Management: While John is highly productive, he sometimes struggles with prioritizing tasks when under tight deadlines. Improving his time management skills could help him handle multiple tasks more efficiently.

Attention to Detail: There have been instances where minor errors were found in his reports and documentation. Paying closer attention to the details will help him maintain accuracy in his work.

Adaptability: John tends to prefer following established procedures, which can sometimes make it challenging for him to adapt to sudden changes or new strategies. Enhancing his flexibility will help him adjust more quickly to evolving market conditions.

Delegation Skills: He occasionally takes on too many tasks himself instead of delegating them to team members. Developing stronger delegation skills will allow John to focus on more strategic activities and increase overall team productivity.'''

In [36]:
result = seq_chain.invoke(input=review)



> Entering new SequentialChain chain...

> Finished chain.


In [40]:
print(result["review"])

Employee Name: John Doe
Position: Sales Representative
Review Period: Q3 2024

Strengths:
Strong Communication Skills: John consistently demonstrates excellent verbal and written communication. His ability to convey ideas clearly to both clients and team members has helped improve client relationships and close sales deals effectively.

Customer-Focused Approach: John shows a deep understanding of customer needs and provides tailored solutions that result in high customer satisfaction. His dedication to addressing customer concerns and following up on issues promptly is commendable.

Goal-Oriented: He is highly motivated and sets clear goals for himself. John has successfully met or exceeded his sales targets in the past three quarters, showing his dedication to achieving measurable results.

Team Collaboration: John works well with his colleagues and contributes positively to team projects. He is always willing to share his knowledge and mentor newer team members, which fosters a supp

## LLM Router Chain

- Can take in an input and re-direct it to the most relevant llm chain sequence
- Does this by adding a description on the input

Information required:

- Name, Description, Template

e.g:

- Student asks a physics doubt
- "How does a magnet work?"
- "Explain Feynman Diagram."

Input --> Router --> LLM Decides Chain --> Chain --> Output

In [42]:
beginner_temp = """You are a physics teacher who is really focused on beginners and explaining complex concepts in simple to understand terms.
You assume no prior knowledge. Here is your questio: \n{input}"""

expert_temp = """You are a physics professor who explains physics topics to advanced audience members. You can assume anyone you answer has a 
PhD in Physics. Here is your question: \n{input}"""

In [43]:
prompt_info = [
    {
        'name': 'beginner physics',
        'description': 'Answer beginner level physics question',
        'template': beginner_temp
    },
    {
        'name': 'advanced physics',
        'description': 'Answer advance level physics questions',
        'template': expert_temp
    }
]

In [44]:
from langchain.chains import LLMChain, RouterChain
from langchain.prompts import ChatPromptTemplate

In [45]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0)

In [46]:
destination_chains = {}

for p_info in prompt_info:

    name = p_info['name']
    prompt_template = p_info["template"]
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [47]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [49]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [54]:
# Setting up destination
destinations = [f"{p['name']}: {p['description']}" for p in prompt_info]
destinations = "\n".join(destinations)
print(destinations)

beginner physics: Answer beginner level physics question
advanced physics: Answer advance level physics questions


In [55]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [57]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics: Answer beginner level physics question
advanced physics: Answer advance level physi

In [58]:
router_prompt = PromptTemplate(template=router_template,
                               input_variables=['input'],
                               output_parser=RouterOutputParser())

In [59]:
from langchain.chains.router import MultiPromptChain

In [60]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [61]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True)

C:\Users\ndh00961\AppData\Local\Temp\ipykernel_29448\1481339893.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [63]:
chain.invoke("Please explain Feynman Diagrams?")



> Entering new MultiPromptChain chain...
advanced physics: {'input': 'Explain Feynman Diagrams in the context of quantum electrodynamics'}
> Finished chain.


{'input': 'Explain Feynman Diagrams in the context of quantum electrodynamics',
 'text': "A delightful topic! Feynman diagrams are a powerful tool for visualizing and calculating the interactions between particles in quantum electrodynamics (QED). As a PhD in Physics, I'm sure you're familiar with the basics of QED, but let me dive deeper into the world of Feynman diagrams.\n\nIn QED, particles interact through the exchange of virtual photons, which are the quanta of the electromagnetic field. These interactions can be represented as a series of vertices, where particles are connected by lines representing the exchange of photons. Feynman diagrams provide a way to organize and calculate these interactions, allowing us to predict the probabilities of different outcomes.\n\nA Feynman diagram typically consists of several components:\n\n1. **Vertices**: These are the points where particles interact with each other or with virtual photons. Vertices are represented by a dot or a cross, and 

## Transform Chain

- Allows us to insert our own functionality into any chain

Input --> Custom Python Transformation --> LLMChain

In [1]:
yelp = open('data\\langchain-course-main\\02-Chains\\yelp_review.txt').read()
print(yelp.split('REVIEW:')[-1].lower())


oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowledgeable about the menu. they went above and beyond

In [31]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts import ChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0)

In [6]:
def transformer_function(inputs: dict) -> dict:

    text = inputs['text']
    only_review_text = text.split("REVIEW:")[-1].lower()
    output = {
        'output': only_review_text
    }

    return output

In [8]:
transform_chain = TransformChain(input_variables=['text'],
                                 output_variables=['output'],
                                 transform = transformer_function)

In [9]:
template = 'Create a one sentence summary of this review:\n{review}'
prompt = ChatPromptTemplate.from_template(template=template)

In [40]:
summary_chain = LLMChain(llm=llm, prompt=prompt, output_key='review_summary')
sequential_chain = SimpleSequentialChain(chains=[transform_chain, summary_chain],
                                         verbose=True)
result_old = sequential_chain.invoke(yelp)



> Entering new SimpleSequentialChain chain...

oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowledg

In [41]:
summary_chain = prompt | llm | StrOutputParser() 
sequential_chain = {"review": transform_chain} | summary_chain
result = sequential_chain.invoke(input={"text": yelp}) 

# Function Calling using LLM

In [45]:
class Scientist:

    def __init__(self, first, last):

        self.first = first
        self.last = last

In [48]:
json_schema = {
                'title': 'Scientist',
                'description': 'Information about a famous scientist',
                'type': 'object',
                'properties': {
                                'first': {
                                            'title': 'First Name',
                                            'description': 'First name of the scientist',
                                            'type': 'string' 
                                         },
                                'last': {
                                            'title': 'Last Name',
                                            'description': 'Last name of a famous scientist',
                                            'type': 'string'
                                        }
                              },
                'required': ['first', 'last']
}

In [49]:
template = "Name a famous {country} scientist"

In [50]:
from langchain_groq import ChatGroq

In [67]:
model = ChatGroq(model='llama3-8b-8192', temperature=0)
prompt = ChatPromptTemplate.from_template(template)
strc_llm = model.with_structured_output(json_schema)

In [74]:
request = prompt.format_prompt(country="Indian")
result = strc_llm.invoke(request)
obj = Scientist(result["first"], result["last"])

In [76]:
obj.last

'V. Raman'

# Math Chain

In [83]:
from langchain import LLMMathChain
math_model = LLMMathChain.from_llm(llm=llm)

In [ ]:
promt = ChatPromptTemplate.from_template(" What is the result of {input}?")
request = promt.format_prompt(input = "17**11")
math_model.invoke(request)

# Additional Chains Available:
Check the additional chains in langchain documentation to check out pre-built chains for common use cases

# Chain Excercise

- Load an email (in spanish in exmaple)
- Detect the language from the email
- Translate to English
- Get a summary of the email
- The function performing this action should return a dictionary. The dictionary should have:
    - Original email
    - Detected language
    - Translated email
    - Summary


In [94]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [107]:
def email_translator(text: str):

    llm = ChatGroq(model='llama3-8b-8192', temperature=0)

    prompt_1 = ChatPromptTemplate.from_template("""Detect the language in which the text {text} is written.
                                                Only return the detected language, nothing else.""")
    chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key='og_lang')

    prompt_2 = ChatPromptTemplate.from_template("""Translate the {text} from {og_lang} to English.
                                                Only return the translated text, nothing else.""")
    chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key='trans_text')

    prompt_3 = ChatPromptTemplate.from_template("""Summarise the text: \n{trans_text}.
                                                Only return the summary, nothing else.""")
    chain_3 = LLMChain(llm=llm, prompt=prompt_3, output_key='summary')

    seq_chain = SequentialChain(chains=[chain_1, chain_2, chain_3],
                                input_variables=["text",],
                                output_variables=["og_lang", "trans_text", "summary"])

    return seq_chain.invoke(text)

In [112]:
text = open("data\langchain-course-main\\02-Chains\spanish_customer_email.txt").read()
email_translator(text)

{'text': 'Asunto: Reporte de Problemas Técnicos - Funcionalidad del Panel SAAS\n\nEstimado Equipo de Soporte al Cliente,\n\nEspero que este mensaje les encuentre bien. Les escribo para informarles sobre un problema técnico que he encontrado mientras utilizo su producto de panel SAAS. Como cliente leal, aprecio el valor que su producto aporta a mi negocio, pero actualmente me enfrento a un desafío que requiere su experiencia.\n\nMe gustaría describir detalladamente el problema que estoy experimentando:\n\n1. Problema Gráfico: Al iniciar sesión en el panel SAAS, he notado que los gráficos y las tablas en la página principal del panel no se renderizan correctamente. Los puntos de datos aparecen distorsionados y algunos elementos se superponen, lo que dificulta la interpretación precisa de la información.\n\n2. Fallo en la Función de Exportación: Además, no he podido exportar informes y datos desde el panel. Cada vez que intento exportar un informe en formato CSV o PDF, recibo un mensaje d